# NiftyPET Example

This is a full demo of NiftyPET's default [OSEM](#OSEM "ordered subsets expectation maximisation") ($n_\text{max}=14$ subsets, span 11, Siemens Biograph mMR resolution), as well as a custom, explicit [MLEM](#MLEM "maximum likelihood expectation maximisation") incorporating [RM](#RM "resolution modelling").


Mathematically:

$$
{\bf\theta}^{(k+1)} = {{\bf\theta}^{(k)} \over \sum_n{{\bf H}^T{\bf X}_n^T{\bf A}^T{\bf N}^T{\bf 1}}}
    \circ
    \sum_n{ {\bf H}^T{\bf X}_n^T{\bf A}^T{\bf N}^T
        { {\bf m} \over {\bf NA}{\bf X}_n{\bf H}{\bf\theta}^{(k)} + {\bf r} + {\bf s} }
    },
$$

- $k$ is iteration number
- $H$ applies a Gaussian PSF
- $X_n$ is the system matrix for subset $n$ (MLEM has just one subset)
- $m, r, s$ are measured, randoms, and scatter

----

- Author: Casper O. da Costa-Luis [casper.dcl@{physics.org|ieee.org|kcl.ac.uk}](mailto:casper.dcl@physics.org)
- Date: 2019-20

----

# Imports

In addition to NiftyPET, this is recommended:
```bash
pip install brainweb[plot]  # image display
pip install spm12  # MATLAB-based registration
```

In [ ]:
# imports
from __future__ import print_function, division
%matplotlib notebook
from collections import OrderedDict
from glob import glob
from os import path
import functools
import logging
import os

if os.getenv("OMP_NUM_THREADS", None) != "1":
    raise EnvironmentError("should run `export OMP_NUM_THREADS=1` before notebook launch")

from brainweb import volshow
from niftypet import nipet
from niftypet.nimpa import getnii
from scipy.ndimage.filters import gaussian_filter
from tqdm.auto import trange
import matplotlib.pyplot as plt
import numpy as np
import pydicom

logging.basicConfig(level=logging.INFO, format=nipet.LOG_FORMAT)
print(nipet.gpuinfo())
# get all the scanner parameters
mMRpars = nipet.get_mmrparams()
# conversion for Gaussian sigma/[voxel] to FWHM/[mm]
SIGMA2FWHMmm = (8 * np.log(2))**0.5 * np.array([mMRpars['Cnt']['SO_VX' + i] for i in 'ZYX']) * 10

def div_nzer(x, y):
    return np.divide(x, y, out=np.zeros_like(y), where=y!=0)

def trimVol(x):
    return x[:, 100:-100, 100:-100]

def norMax(x, scale=1):
    x = trimVol(x).transpose(2, 0, 1)[..., None]
    return x * (scale / x.max())

def register_spm(ref_file, mov_file, opth):
    """
    ref_file  : e.g. recon['fpet']
    mov_file  : e.g. datain['T1nii']
    """
    from spm12.regseg import coreg_spm, resample_spm
    reg = coreg_spm(ref_file, mov_file, outpath=opth, save_arr=False, save_txt=False)
    return getnii(resample_spm(ref_file, mov_file, reg['affine'], outpath=opth,
                               del_ref_uncmpr=True, del_flo_uncmpr=True, del_out_uncmpr=True))

def register_dipy(ref_file, mov_file, ROI=None):
    """
    ref_file  : e.g. recon['fpet']
    mov_file  : e.g. datain['T1nii']
    """
    from brainweb import register
    return register(getnii(mov_file), getnii(ref_file), ROI=ROI or ((0, None), (100, -100), (100, -100)))

# Load & Process Raw Data

In [ ]:
folderin = "amyloidPET_FBP_TP0"
folderout = "."  # realtive to `{folderin}/niftyout`
itr = 7  # number of iterations (will be multiplied by 14 for MLEM)
fwhm = 2.5  # mm (for resolution modelling)
totCnt = None  # bootstrap sample (e.g. `300e6`) counts

In [ ]:
# datain
folderin = path.expanduser(folderin)

# automatically categorise the input data
#logging.getLogger().setLevel(logging.INFO)
datain = nipet.classify_input(folderin, mMRpars, recurse=-1)

# output path
opth = path.join(datain['corepath'], "niftyout")
# switch on verbose mode
#logging.getLogger().setLevel(logging.DEBUG)

datain

In [ ]:
# hardware mu-map (bed, head/neck coils)
mu_h = nipet.hdw_mumap(datain, [1,2,4], mMRpars, outpath=opth, use_stored=True)

In [ ]:
# MR-based human mu-map

# UTE-based object mu-map aligned (need UTE sequence or T1 for pseudo-CT)
#mu_o = nipet.align_mumap(
#    datain,
#    scanner_params=mMRpars,
#    outpath=opth,
#    t0=0, t1=0, # when both times are 0, will use full data
#    itr=2,      # number of iterations used for recon to which registering MR/UTE
#    petopt='ac',# what PET image to use (ac-just attenuation corrected)
#    musrc='ute',# source of mu-map (ute/pct)
#    ute_name='UTE2', # which UTE to use (UTE1/2 shorter/faster)
#    verbose=True,
#)

#> the same as above without any faff though (no alignment)
mu_o = nipet.obj_mumap(datain, mMRpars, outpath=opth, store=True)

In [ ]:
# create histogram
mMRpars['Cnt']['BTP'] = 0
m = nipet.mmrhist(datain, mMRpars, outpath=opth, store=True, use_stored=True)
if totCnt:
    mMRpars['Cnt']['BTP'] = 2  # enable parametric bootstrap
    mMRpars['Cnt']['BTPRT'] = totCnt / m['psino'].sum()  # ratio count level relative to the original
    m = nipet.mmrhist(datain, mMRpars, outpath=path.join(opth, 'BTP', '%.3g' % totCnt), store=True)

## Visualisations

In [ ]:
try:  # needs HW mu-maps
    volshow(mu_o['im'] + mu_h['im'], cmaps=['bone'], titles=[r"$\mu$-map"])
except:
    volshow(mu_o['im'], cmaps=['bone'])

In [ ]:
# sinogram index (<127 for direct sinograms, >=127 for oblique sinograms)
volshow([m['psino'], m['dsino']],
        titles=["Prompt sinogram (%.3gM)" % (m['psino'].sum() / 1e6),
               "Delayed sinogram (%.3gM)" % (m['dsino'].sum() / 1e6)],
        cmaps=['inferno'] * 2, xlabels=["", "bins"], ylabels=["angles"] * 2, ncols=2, colorbars=[1]*2,
       figsize=(9.5, 3.5), tight_layout=5);

# Reconstruction

## OSEM

In [ ]:
# built-in default: 14 subsets
fcomment = f"_fwhm-{fwhm}_recon"
outpath = path.join(opth, folderout)
recon = glob(
    f"{outpath}/PET/single-frame/a_t-*-*sec_itr-{itr}{fcomment}.nii.gz"
)
if recon:
    recon = {'fpet': recon[0], 'im': getnii(recon[0])}
else:
    recon = nipet.mmrchain(
        datain, mMRpars,
        #frames=['timings', [3000, 3600]],
        itr=itr,
        #store_itr=range(itr),
        histo=m,
        mu_h=mu_h,
        mu_o=mu_o,
        fwhm_rm=fwhm,
        #recmod=1,  # no scatter & rand
        outpath=outpath,
        fcomment=fcomment,
        store_img=True)

volshow([trimVol(recon['im'])], cmaps=['magma']);

### Registration

In [ ]:
# convert DCM to NII if required
if 'T1nii' not in datain:
    !{mMRpars['Cnt']['DCM2NIIX']} -z y -f 'T1' {datain['T1DCM']}
    extra = nipet.classify_input(datain['T1DCM'], mMRpars)
    extra.pop('corepath')
    datain.update(extra)

#filepaths = sorted(glob(path.join(datain['T1DCM'], '*.IMA')))
#arr = np.asanyarray([pydicom.dcmread(i).pixel_array for i in filepaths]).transpose(1, 2, 0)

In [ ]:
# SPM12
regout = f"{opth}/affine_ref-a_t-*sec_itr-21_fwhm-4.5_recon.nii.gz"
regim = getnii(regout) if path.exists(regout) else register_spm(recon['fpet'], datain['T1nii'], opth)

# Simple Alternative: DIPY CoM
#regim = register_dipy(recon['fpet'], datain['T1nii'])

### Figures

In [ ]:
volshow({
    "PET": trimVol(recon['im']), #.transpose(2, 0, 1),
    "Smoothed": gaussian_filter(trimVol(recon['im']), 4.5 / SIGMA2FWHMmm), #.transpose(2, 0, 1),
    "$\\mu$ Map": trimVol(mu_o['im']), #.transpose(2, 0, 1),
    "T1": trimVol(regim), #.transpose(2, 0, 1)
},
vmaxs=[np.percentile(recon['im'], 99.95)]*2 + [None]*2,
cmaps=['magma', 'magma', 'Greys', 'Greys_r'], nrows=1, figsize=(9.5, 3.5));

In [ ]:
# rgb => PET, mu_o, T1
volshow([np.concatenate([norMax(recon['im']), norMax(mu_o['im'], 0.15), norMax(regim, 0.75)], axis=-1)]);

## MLEM

In [ ]:
## Attenuation, Normalisation & Sensitivity

A = nipet.frwd_prj(mu_h['im'] + mu_o['im'], mMRpars, attenuation=True)
N = nipet.mmrnorm.get_norm_sino(datain, mMRpars, m)
AN = A * N
sim = nipet.back_prj(AN, mMRpars)
msk = nipet.img.mmrimg.get_cylinder(mMRpars['Cnt'], rad=29., xo=0., yo=0., unival=1, gpu_dim=False) <= 0.9

## Randoms

r = nipet.randoms(m, mMRpars)[0]
print("Randoms: %.3g%%" % (r.sum() / m['psino'].sum() * 100))

## Scatter

# One OSEM iteration estimate (implicitly using voxel-driven scatter model)
eim = nipet.mmrchain(datain, mMRpars, mu_h=mu_h, mu_o=mu_o, histo=m, itr=1, outpath=opth)['im']
# Recalculate scatter
s = nipet.vsm(datain, (mu_h['im'], mu_o['im']), eim, m, r, mMRpars)
print("Scatter: %.3g%%" % (s.sum() / m['psino'].sum() * 100))

In [ ]:
volshow(OrderedDict([("Prompts", m['psino']), ("Delayed", m['dsino']), ("Attenuation", A),
                     ("Scatter", s), ("Randoms", r), ("Normalisation", N)]),
        cmaps=['inferno']*6, colorbars=[1]*6, ncols=3, figsize=(9.5, 6));

In [ ]:
## MLEM with RM

psf = functools.partial(gaussian_filter, sigma=fwhm / SIGMA2FWHMmm)
sim_inv = div_nzer(1, psf(sim))
sim_inv[msk] = 0
rs_AN = div_nzer(r + s, AN)
recon_mlem = [np.ones_like(sim)]
for k in trange(itr * 14, desc="MLEM"):
    fprj = nipet.frwd_prj(psf(recon_mlem[-1]), mMRpars) + rs_AN
    recon_mlem.append(recon_mlem[-1] * sim_inv * psf(nipet.back_prj(div_nzer(m['psino'], fprj), mMRpars)))

In [ ]:
# central slice across iterations
volshow(
    np.asanyarray(recon_mlem[14::14])[:, :, 90:-100, 110:-110],
    cmaps=['magma'] * len(recon_mlem[14::14]), ncols=4, figsize=(9.5, 7), frameon=False);